In [ ]:
# Necessary imports
import os
import pathlib
os.environ['USE_PYGEOS'] = '0'
import gc
import numpy as np
from itertools import product
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors
import ee

import pandas as pd
from georeader.readers import ee_query
import folium
import geemap.foliumap as geemap
import shapely
from shapely.geometry import box, Point, MultiPoint
from shapely.ops import nearest_points

from skimage.morphology import binary_dilation, disk

from typing import Tuple, Callable, Any, Optional


from ml4floods.data.worldfloods import dataset
from ml4floods.data import utils
from ml4floods.visualization import plot_utils
from ml4floods.models import postprocess

from db_utils import DB

# Uncomment this to suppress deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

# Set bucket will not be requester pays
utils.REQUESTER_PAYS_DEFAULT = False

In [ ]:
import rasterio
from rasterio.io import MemoryFile
from rasterio import Affine as A
#from rasterio.warp import reproject, Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_origin


## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"

print("[INFO] Successfully loaded FloodMapper environment.")

In [ ]:
# Connect to the database (point to the .env file for credentials)
db_conn = DB(env_file_path)

In [ ]:
# Initialise the Google Earth Engine connection.
# Follow instructions on login prompt, if required.
ee.Initialize()

In [ ]:
# EDIT THIS CELL: All work is conducted under a unique session name
session_name = "boulia_test"

## Retrieve the session parameters from the database

Edit the following cell to set the session name and retrieve the flood map parameters, including mapping grid and affected LGAs.

In [ ]:
tz = ZoneInfo("UTC")
midnight = datetime.min.time()

# Query the floodmapping parameters from the DB
query = (f"SELECT flood_date_start, flood_date_end, "
         f"ref_date_start, ref_date_end, bucket_uri "
         f"FROM session_info "
         f"WHERE session = %s;")
data = (session_name,)
session_df = db_conn.run_query(query, data, fetch=True)
flood_start_date = session_df.iloc[0]["flood_date_start"]
flood_start_date = datetime.combine(flood_start_date, midnight).replace(tzinfo=tz)
flood_end_date = session_df.iloc[0]["flood_date_end"]
flood_end_date = datetime.combine(flood_end_date, midnight).replace(tzinfo=tz)
ref_start_date = session_df.iloc[0]["ref_date_start"]
ref_start_date = datetime.combine(ref_start_date, midnight).replace(tzinfo=tz)
ref_end_date = session_df.iloc[0]["ref_date_end"]
ref_end_date = datetime.combine(ref_end_date, midnight).replace(tzinfo=tz)
bucket_uri = session_df.iloc[0]["bucket_uri"]

# Query the selected grid positions and LGAs
query = (f"SELECT sp.patch_name, ST_AsText(gr.geometry), gr.lga_name22 "
         f"FROM session_patches sp "
         f"INNER JOIN grid_loc gr "
         f"ON sp.patch_name = gr.patch_name "
         f"WHERE sp.session = %s ;")
data = (session_name,)
grid_sel_df = db_conn.run_query(query, data, fetch=True)

# Format the results into a correct GeoDataFrame
grid_sel_df['geometry'] = gpd.GeoSeries.from_wkt(grid_sel_df['st_astext'])
grid_sel_df.drop(['st_astext'], axis=1, inplace = True)
grid_sel_gdf = gpd.GeoDataFrame(grid_sel_df, geometry='geometry', crs="EPSG:4326")
grid_sel_gdf.drop_duplicates(subset=["patch_name"], inplace=True)
print(f"[INFO] {len(grid_sel_gdf)} grid patches selected.")

# Query the affected LGA shapes
lgas_sel_lst = grid_sel_df.lga_name22.unique().tolist()
query = (f"SELECT DISTINCT lga_name22, ST_AsText(geometry_col) "
         f"FROM lgas_info "
         f"WHERE lga_name22 IN %s ;")
data = (tuple(lgas_sel_lst),)
lgas_sel_df = db_conn.run_query(query, data, fetch=True)

# Format the results into a correct GeoDataFrame
lgas_sel_df['geometry'] = gpd.GeoSeries.from_wkt(lgas_sel_df['st_astext'])
lgas_sel_df.drop(['st_astext'], axis=1, inplace = True)
lgas_sel_gdf = gpd.GeoDataFrame(lgas_sel_df, geometry='geometry', crs="EPSG:4326")
print(f"[INFO] {len(lgas_sel_gdf)} LGAs affected.")

## Problem patches

We will use the following patches during development and testing:

* GRID22075
* GRID22244
* GRID21909
* GRID21910

In [ ]:
# Set the pactch to be processed
#patch_name = "GRID22075"
#patch_name = "GRID21910"
patch_name = "GRID21909"
#patch_name = "GRID22412"

# Select the list of flood-maps during the time range
# Order by satellite so as S2 are first
query = (f"SELECT DISTINCT satellite, date, data_path "
         f"FROM inference "
         f"WHERE patch_name = %s "
         f"AND date >= %s "
         f"AND date <= %s "
         f"AND mode = %s "
         f"AND status = %s "
         f"ORDER BY satellite DESC, date ASC")
data = [patch_name, flood_start_date, flood_end_date, 'pred', 1]
geojsons_df = db_conn.run_query(query, data, fetch=True)
num_files = len(geojsons_df)
print(f"[INFO] Found {num_files} flood maps in the database.")
geojsons_df

## Final function for performing temporal aggregation

The function get_floodmap_post() expects a list of paths to vector GeoJSONs. We will build a function that expects a list of paths to raster GeoJSONs and perform the vectorisation step after.

In [ ]:
# Extract the list of GeoJSONs from the dataframe.
# Sort so that Sentinel-2 is the first file in the list
geojsons_lst = geojsons_df["data_path"].tolist()

In [ ]:
# Define the vectorise function
def vectorize_outputv1(prediction: np.ndarray,
                       crs: Any,
                       transform: rasterio.Affine,
                       border:int=2) -> Optional[gpd.GeoDataFrame]:
    """
    Convert a raster mask into a vectorised GeoDataFrame.

    Args:
        prediction: (H, W) array with 4 posible values [0: "invalid",
                    2: "water", 3: "cloud", 4: "flood_trace"]
        crs:        coordinate reference system
        transform:  transformation matrix
        border:     set border pixels to zero

    Returns:
        GeoDataFrame with vectorised masks
    """
    data_out = []
    start = 0
    class_name = {0: "area_imaged", 2: "water", 3: "cloud", 4: "flood_trace"}
    # Dilate invalid mask
    invalid_mask = binary_dilation(prediction == 0, disk(3)).astype(bool)

    # Set borders to zero to avoid border effects when vectorizing
    prediction[:border,:] = 0
    prediction[:, :border] = 0
    prediction[-border:, :] = 0
    prediction[:, -border:] = 0
    prediction[invalid_mask] = 0

    # Loop through the mask classes
    for c, cn in class_name.items():
        if c == 0:
            # To remove stripes in area imaged
            mask = prediction != c
        else:
            mask = prediction == c

        geoms_polygons = \
            postprocess.get_water_polygons(mask, transform=transform)
        if len(geoms_polygons) > 0:
            data_out.append(gpd.GeoDataFrame(
                {"geometry": geoms_polygons,
                 "id": np.arange(start, start + len(geoms_polygons)),
                 "class": cn},
                crs=crs))
        start += len(geoms_polygons)

    if len(data_out) == 1:
        return data_out[0]
    elif len(data_out) > 1:
        return pd.concat(data_out, ignore_index=True)

    return None

In [ ]:
def calc_maximal_floodraster(geojsons_lst, verbose=False):
    """
    Calculate the maximal flood extent from the integer-based 
    raster flood masks.  
    """

    is_first = True
    geojsons_lst.sort(reverse=True) # Sort so that S2 is first
    for filename in geojsons_lst:
        if verbose:
            sat_file = "_".join(pathlib.Path(filename).parts[-2:])
            print(f"[INFO] temporal merge '{sat_file}'")
        with rasterio.open(filename) as src:
            if is_first:
                is_first = False
                # Read the header and raster array
                profile = src.profile.copy()
                band1 = src.read(1)
                # Record the target CRS and dimensions
                dst_crs = src.crs
                dst_width = src.width
                dst_height = src.height
                dst_bounds = src.bounds
                # Create the initial masks
                valid = band1 != 0
                water = band1 == 2
                cloud = band1 == 3
                flood_trace = band1 == 4
            else:
                # Calculate the output transformation matrix
                dst_transform, dst_width, dst_height = \
                calculate_default_transform(
                    src.crs,
                    dst_crs,
                    dst_width,
                    dst_height,
                    *dst_bounds)
                # Build a new header
                dst_kwargs = src.meta.copy()
                dst_kwargs.update({
                    'crs': dst_crs,
                    'transform': dst_transform,
                    'width': dst_width,
                    'height': dst_height,
                    'nodata': 0
                })
                # Perform operations in memory
                with MemoryFile() as memfile:
                    # Reproject band 1 to a memory file
                    with memfile.open(**dst_kwargs) as dst:
                        reproject(
                            source=rasterio.band(src, 1),
                            destination=rasterio.band(dst, 1),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=dst_transform,
                            dst_crs=dst_crs,
                            resampling=Resampling.nearest)
                            #resampling=Resampling.bilinear)
                    # Accumulate the masks onto the final arrays
                    with memfile.open() as mch:
                        band1 = mch.read(1)
                        # Water always accumulates into a maximum extent
                        water += (band1 == 2)
                        # Flood_trace accumulates, except where it converts to water
                        flood_trace += (band1 == 4)
                        flood_trace = np.where(water, False, flood_trace)
                        # Cloud accumulates, but is nulified by water, flood_trace and land.
                        # Aim is to only have cloud masks where no data exists because of clouds.
                        cloud += (band1 == 3)
                        cloud = np.where(water, False, cloud)
                        cloud = np.where(flood_trace, False, cloud)
                        land = (band1 != 0) & (band1 != 2) & (band1 != 3) & (band1 != 4)
                        cloud = np.where(land, False, cloud)
                        # Valid data accumulates as a maximum extent
                        valid += (band1 != 0)
            
        plt.imshow(water)
        plt.show()
        
    # Assemble the final array
    out_raster = np.zeros_like(band1)
    out_raster = np.where(valid, 1, out_raster)
    out_raster = np.where(cloud, 3, out_raster)
    out_raster = np.where(flood_trace, 4, out_raster)
    out_raster = np.where(water, 2, out_raster)
    
    # Vectorise the output
    floodmap = vectorize_outputv1(out_raster, profile['crs'], profile['transform'])

    return out_raster, profile, floodmap

In [ ]:
# Call the maximal map calculator and plot
raster, profile, floodmap = calc_maximal_floodraster(geojsons_lst, True)
plt.imshow(raster)
plt.show()

In [ ]:

# Plot the floodmap
plot_utils.plot_floodmap(floodmap) 

In [ ]:
profile

https://gis.stackexchange.com/questions/279953/numpy-array-to-gtiff-using-rasterio-without-source-raster

In [ ]:
raster.shape